### instalar lib

In [17]:
import json
import os
from datetime import datetime
import  yfinance as yf
from crewai import Agent, Task
from langchain.tools import Tool
import pandas as pd

In [18]:
def fetch_stock_price(ticker_input):
    # Extract just the ticker symbol if it's provided in a more complex format
    if isinstance(ticker_input, dict) and 'ticker' in ticker_input:
        ticker = ticker_input['ticker']
    elif isinstance(ticker_input, str):
        ticker = ticker_input.strip().upper()
    else:
        raise ValueError("Invalid ticker format. Please provide a valid ticker symbol.")
    
    # Validate the ticker before downloading
    if not ticker or not isinstance(ticker, str):
        raise ValueError("Ticker symbol is required and must be a string.")
    
    try:
        # Use a more reliable date range and add error handling
        stock = yf.download(ticker, start='2023-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))
        
        # Check if data was actually returned
        if stock.empty:
            return f"No data found for ticker {ticker}. Please verify the ticker symbol is correct."
        
        return stock
    except Exception as e:
        return f"Error fetching data for {ticker}: {str(e)}"

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="This tool fetches stock price information for a given ticker symbol from Yahoo Finance",
    func=fetch_stock_price,
)

In [19]:
# Test the updated tool
response = yahoo_finance_tool.run("AAPL")
print(response)

# Test with ticker in dict format
# response = yahoo_finance_tool.run({"ticker": "MSFT"})
# print(f"Data for MSFT:\n{response.tail() if not isinstance(response, str) else response}")

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2023-01-03  123.632530  129.395518  122.742873  128.782649  112117500
2023-01-04  124.907700  127.181268  123.642412  125.431607   89113600
2023-01-05  123.583099  126.301493  123.326093  125.668849   80962700
2023-01-06  128.130234  128.792531  123.454601  124.561732   87754700
2023-01-09  128.654129  131.876670  128.397123  128.970458   70790800
...                ...         ...         ...         ...        ...
2025-03-10  227.479996  236.160004  224.220001  235.539993   72071200
2025-03-11  220.839996  225.839996  217.449997  223.809998   76137400
2025-03-12  216.979996  221.750000  214.910004  220.139999   62547500
2025-03-13  209.679993  216.839996  208.419998  215.949997   61368300
2025-03-14  213.490005  213.949997  209.580002  211.250000   60060200

[551 rows x 5 colum

In [ ]:
# Advanced version returning structured data
def fetch_stock_data_structured(ticker_input):
    # Get the raw data using our existing function
    result = fetch_stock_price(ticker_input)
    
    # If the result is a string, it's an error message
    if isinstance(result, str):
        return result
        
    # Structure the data
    return {
        "ticker": ticker_input if isinstance(ticker_input, str) else ticker_input.get('ticker'),
        "data_points": len(result),
        "date_range": f"{result.index.min().strftime('%Y-%m-%d')} to {result.index.max().strftime('%Y-%m-%d')}",
        "latest_close": result['Close'].iloc[-1] if not result.empty else None,
        "latest_data": result.tail(5).to_dict()
    }

# Create an alternative tool that returns structured data
yahoo_finance_structured_tool = Tool(
    name="Yahoo Finance Structured Tool",
    description="This tool fetches structured stock price information for a given ticker symbol from Yahoo Finance",
    func=fetch_stock_data_structured,
)